# ImpactChron.jl Tutorial

In this brief tutorial, I outline some of the main functionality in ImpactChron.jl. For more detailed information about the package, check out its [documentation](https://grahamedwards.github.io/ImpactChron.jl/dev/).

Now, let's get to business. First, load the package...

In [ ]:
using ImpactChron

### Time Management
Now, let's declare the time range and resolution we'll use, including: 
- The age of the solar system --- `solarage` --- will be the start-time of our model. Let's set that to the age of CAIs.
- The youngest age the model runs to --- `minage` --- will be the stop-time of our model. Let's set that to 2 Ga.
- The timestep of our model is `Δt`.
- To smooth the data for statistical analysis, we'll bin our distribution of ages with a 1 My timestep into 10 My bins by setting `downscalefactor=10`.
- Finally, let's set a forward-time limit --- `timelimit` --- for the model that spans `solarage` to `minage` and is divisible by  `downscalefactor`. 

In [ ]:
solarage = 4567.3 # Ma
minage = 2000. # Ma
Δt = 1. # My
downscalefactor = 10
timelimit = 2569.; # My after CAIs

### Measured ages
Next, let's load in our database of Ar-Ar ages, and only include ages > `minage`.

In [ ]:
priordata = ImpactChron.readdlm("data/ArArages.csv",',')
ages_ = priordata[:,4]
iages = ages_ .> minage
ages = (mu = float.(ages_[iages]), sig = float.(priordata[:,5])[iages]);

### Weighting by petrologic type
Many of us meteoriticists (or maybe just me...) lose sleep fretting over the reality that the meteorite record is likely a biased sample of asteroid interiors. Even worse, we heterogenously sample this meteorite record in our selections for chronometric analyses. 

We do our best to account for this in ImpactChron. Since petrologic types are a reasonable proxy for depth in parent planetesimals (e.g. [Miyamoto 1982](https://ui.adsabs.harvard.edu/abs/1982LPSC...12.1145M/abstract)), we'll reweight the abundances of modeled Ar-Ar ages so that their proportions of petrologic types match the proportions of petrologic types in our database of measured Ar-Ar ages.

In [ ]:
types = PetroTypes((T3=600+273.,T4=700+273.,T5=800+273.,T6=950+273.), string.(priordata[iages,3]));

### The shape of regions reheated by impacts
We assume a probabilistic volume of reheating in the shape of a cone that extends to a point at the asteroid center and has a base diameter that occupies 1% of the asteroid's circumference. (More in forthcoming pre-print)

Don't like this approach? No problem, you can give `ImpactSite` the shape of a `Hemisphere` or `Parabola` with size calculated relative to an impactor diameter following the proportions in [Davison+ 2012](http://doi.org/10.1016/j.gca.2012.08.001), e.g. `ImpactSite(Parabola,8e3)` for a paraboloid reheating zone produced by an 8 km impactor.

In [ ]:
impactsite=ImpactSite(Cone,C=0.01);

### Variables, their distributions, and initial guesses...

In the following cell, we'll declare guesses for each parameter that goes into the asteroid-scale thermochronologic code. These will be wrapped in a NamedTuple called `φ`. The table below defines the field names for these parameters and identifies whether the corresponding values are lognormally distributed and require a natural-logarithm-space guess.

In addition to `φ`, we'll create two more NamedTuples with the same fields:
1.  `φσ` provides intial scaling factors for the symmetric (Gaussian) jump of the Metropolis algorithm.
2.  `distributions` defines the parametric prior distributions of our variables, using the custom types `Nrm`, `lNrm`, and `Unf`, which respectively denote normal, log-normal, and uniform distributions.

Finally, let's declare the variables we want the Bayesian inversion to explore by listing their corresponding field names (as Symbols) in the Tuple `vars`. 

In this example, we'll explore a two-bombardment impact history (note that `Fχγ=0.` and all `:Fχ_` are excluded from `vars`). The α impact flux is primordial, so `tχα` is held at 0 Myₛₛ. The solar system age is a constant in our model framework and should never be included in `vars`. The Bayesian inversion will explore all other parameters. 

#### Parameters:

| Description               | log?  | `field`|
| :------------------------ | :--:  | :----: |
| solar system age (Ma)     | no    | `tss`  |
| initial ²⁶Al/²⁷Al         | no    | `rAlo` |
| closure temperature (K)   | yes   | `Tc`   | 
| body radius (m)           | yes   | `R`    | 
| accretion date (Myₛₛ)      | yes   | `ta`   | 
| disk temperature (K)      | yes   | `Tm`   |
| Al abundance (g/g)        | yes   | `cAl`  |
| density (kg/m³)           | yes   | `ρ`    | 
| thermal diffusivity  (W/[m K])     | yes   | `k`    | 
| specific heat capacity (J/[kg K])   | yes   | `Cp`   |
| α bombardment onset (Myₛₛ) | no    | `tχα`  | 
| α initial flux (My⁻¹)     | no    | `Fχα`  | 
| α ℯ-folding time (My)     | no    | `τχα`  | 
| β bombardment onset (Myₛₛ) | no    | `tχβ`  | 
| β initial flux (My⁻¹)     | no    | `Fχβ`  | 
| β ℯ-folding time (My)     | no    | `τχβ`  | 
| γ bombardment onset (Myₛₛ) | no    | `tχγ`  | 
| γ initial flux (My⁻¹)     | no    | `Fχγ`  | 
| γ ℯ-folding time (My)     | no    | `τχγ`  | 

(Myₛₛ denotes My after CAIs or 4567.3 Ma)

In [ ]:
ϕ = ( tss=solarage, rAlo=5.23e-5, Tm=log(250), R=log(150e3), ta=log(2.0), cAl=log(0.011), ρ=log(3210), Cp=log(900), k=log(3), Tc=log(500),
    tχα=0., τχα=10., Fχα=10., 
    tχβ=5., τχβ=10., Fχβ=10., 
    tχγ=0., τχγ=10., Fχγ=0.)

ϕσ = ( tss=.08, rAlo=0.065e-5, Tm=0.47,R=0.16, ta=.07, cAl=0.13, ρ=0.05, Cp=0.08, k=0.6, Tc=0.2, tχα=1., τχα=1., Fχα=1., tχβ=1., τχβ=1., Fχβ=1., tχγ=1., τχγ=1., Fχγ=1. )

distributions = (
    tss = Nrm(4567.3,0.08), rAlo= Nrm(5.23e-5,0.065e-5), R = lNrm(11.920, 0.1869), ta = lNrm(0.6983, 0.0792), cAl = lNrm(-4.5665, 0.1316), Tm = lNrm(5.3517, 0.4691), Tc = lNrm(6.1914, 0.2571), ρ = lNrm(8.1199, 0.03996), Cp = lNrm(6.732, 0.07899), k = lNrm(0.3319, 0.6335),
    tχα  = Unf(0.,ϕ.tss-minage), τχα  = Unf(0.,ϕ.tss), Fχα  = Unf(0.,1e4),
    tχβ  = Unf(0.,ϕ.tss-minage), τχβ  = Unf(0.,ϕ.tss), Fχβ  = Unf(0.,1e4), 
    tχγ  = Unf(0.,ϕ.tss-minage), τχγ  = Unf(0.,ϕ.tss), Fχγ  = Unf(0.,1e4));

vars = (:rAlo,:R,:ta,:cAl,:Tm,:Tc,:ρ,:Cp,:k,:τχα,:Fχα,:tχβ,:τχβ,:Fχβ);

### Metropolis function

Finally, we'll run the Metropolis algorithm with the variables declared above. For brevity, this illustration is only recording 10 steps of Markov chains (`markovchainsteps`) after a 10 step `burnin`. It will print out updates as it runs, including letting you know everytime a proposal is rejected for over-melting the planetesimal or missing petrologic types. A few extra parameters include the minimum temperature to register a cooling age (`Tmin = 0` K), the maximum temperature (`Tmax = 1373` K), and the number of simulated radial nodes in the model asteroid (`nᵣ = 200`).

The function returns a Dict of Markov chains for each parameter in `φ`. I've named it `markovchains` so you can use it for further exploration, e.g. `markovchainsteps[:R]`.

In [ ]:
burnin, markovchainsteps = 10, 10

markovchains = thermochron_metropolis(ϕ, ϕσ, vars, ages.mu, ages.sig, impactsite, plims=distributions, petrotypes=types, burnin=burnin, nsteps=markovchainsteps, Δt=Δt, downscale=downscalefactor, Tmin=0., Tmax=1373., tmax=timelimit, nᵣ=200)

---
---

## Appendix: underlying thermal codes
### Primary planetesimal cooling
The function below calculates the Ar-Ar ages (in `ArArage`) at radial depths in `z` (in m from asteroid center) given the input parameters contained in `φ` and kwargs (keyword arguments) as described in the cells above. The function also returns the `volumetricfraction` and `peaktemperature` of each radial shell with radial midpoints in `z`.

In [ ]:
ArArage, volumetricfraction, z, peaktemperature = planetesimal_cooling_dates(ϕ, nᵣ=10, Δt=Δt, Tmax=1373, Tmin=0);

### Adding an impact history

The cell above only calculates an unperturbed thermochronologic history (no impacts). We superimpose the effect of impacts onto this history using the function `ImpactChron.impact_reset_array!`. The function `asteroid_agedist!` is a wrapper around a `planetesimal_cooling_` function and this impact-resetting function. The custom type `AsteroidHistory` contains all the relevant information that is used (and reused) within `thermochron_metropolis`. 

In this example, we initialize an `AsteroidHistory` instance, and overwite it with `asteroid_agedist!` using the parameter guesses in `φ`. Rather than show all the complicated data in here, I'm just reporting the most abundant age in the body. If you wish to really explore the data, I recommend running these functions locally and using a plotting package to visualize the results!

In [ ]:
asteroid = AsteroidHistory(ϕ.tss, nnodes=200, Δt=Δt, tmax=timelimit, downscale_factor=10)

asteroid_agedist!(asteroid, ϕ, types, impactsite, nᵣ=200,Tmax=1373,Tmin=0)

mostabundantage = ϕ.tss - asteroid.t_downscaled[findmax(asteroid.agedist_downscaled)[2]]

"The most abundant age in the model asteroid is $mostabundantage Ma"